In [ ]:
def get_performance_stats_2(signal, returns, market_cap, label=None, model_type=None):
    """
    Generate the long/short, long only and short only performance stats for a time-series price data.

    :param signal: pd.dataframe, a panel dataframe with monthly buying (1), selling (-1),  or doing nothing (0) signal
                   for several assets (as columns) over an arbitrary timespan (as index).
    :param returns: pd.dataframe, a panel dataframe with several assets' monthly returns over an arbitrary timespan.
    :param market_cap: pd.dataframe, a panel dataframe with several assets' monthly market cap over an arbitrary timespan.
    :param label: strings, column label as string
    :param model_type: boolean or str, it can be None, 'ML', 'LSTM' or '13'.
    :return: stats - pd.dataframe with statistics (performance measures) organized by row with exact labels:
             Total Return           - total cumulative return over the whole period
             Avg Monthly Return     - average monthly return over the whole period
             Avg Mkt Cap            - average market cap for the chosen stocks
             Fraction>0             - the fraction among chosen stocks that generates positive returns
             + Return Stocks (Avg)  - average return for the chosen stocks having positive returns
             - Return Stocks (Avg)  - average return for the chosen stocks having negative returns
             Odds Ratio             - the fraction that the portfolio of chosen stocks generating positive return over the whole period
             Odds Ratio 98-10       - the fraction that the portfolio of chosen stocks generating positive return from 1998 to 2010
             Odds Ratio 11-22       - the fraction that the portfolio of chosen stocks generating positive return from 2011 to 2022
             Transactions/Month     - average transactions made per month
    """

    signal.index = signal.index.astype('str')
    returns.index = returns.index.astype('str')
    market_cap.index = market_cap.index.astype('str')

    # Pre-allocate empty dataframe
    if label is None:
        stats1 = pd.DataFrame(index=['(Long & Short)'])
        stats2 = pd.DataFrame(index=['(Long Position)'])
        stats3 = pd.DataFrame(index=['(Short Position)'])
    else:
        stats1 = pd.DataFrame(index=[label + ' (Long & Short)'])
        stats2 = pd.DataFrame(index=['(Long Position)'])
        stats3 = pd.DataFrame(index=['(Short Position)'])


    long_short_positions = signal
    long_positions = signal.replace(-1, 0)
    short_positions = signal.replace(1, 0)

    long_short_returns = (signal * returns).replace(-0, 0)
    long_short_sum = long_short_returns.sum(axis=1)
    long_short_returns_98_10 = long_short_returns[:152]
    long_short_returns_11_22 = long_short_returns[152:]
    long_returns = (long_positions * returns).replace(-0, 0)
    long_sum = long_returns.sum(axis=1)
    long_returns_98_10 = long_returns[:152]
    long_returns_11_22 = long_returns[152:]
    short_returns = (short_positions * returns).replace(-0, 0)
    short_sum = short_returns.sum(axis=1)
    short_returns_98_10 = short_returns[:152]
    short_returns_11_22 = short_returns[152:]


    df_transactions = pd.DataFrame({'1998-05':[abs(long_short_positions.iloc[0]).sum()]})

    for l in range(len(long_short_positions.index[:-1])):
        df_2months = long_short_positions[l:l+2].T
        df_2months['diff'] = df_2months.iloc[:,1] - df_2months.iloc[:,0]
        turnover = np.count_nonzero(df_2months['diff'])
        df_transactions = pd.concat([df_transactions, pd.DataFrame({long_short_positions.index[l+1]: [turnover]})], axis=1)


    df_transactions_1 = pd.DataFrame({'1998-05':[abs(long_positions.iloc[0]).sum()]})

    for l in range(len(long_positions.index[:-1])):
        df_2months = long_positions[l:l+2].T
        df_2months['diff'] = df_2months.iloc[:,1] - df_2months.iloc[:,0]
        turnover = np.count_nonzero(df_2months['diff'])
        df_transactions_1 = pd.concat([df_transactions_1, pd.DataFrame({long_positions.index[l+1]: [turnover]})], axis=1)


    df_transactions_2 = pd.DataFrame({'1998-05':[abs(short_positions.iloc[0]).sum()]})

    for l in range(len(short_positions.index[:-1])):
        df_2months = short_positions[l:l+2].T
        df_2months['diff'] = df_2months.iloc[:,1] - df_2months.iloc[:,0]
        turnover = np.count_nonzero(df_2months['diff'])
        df_transactions_2 = pd.concat([df_transactions_2, pd.DataFrame({short_positions.index[l+1]: [turnover]})], axis=1)


    if model_type is None:
        prices1 = get_price_df(long_short_positions, returns, model_type=None)
        last_index1 = prices1.shape[0] - 1
        p_last1 = prices1[last_index1]
        stats1['Total Return'] = (p_last1 - prices1[0]) / prices1[0]
    elif model_type == 'ML':
        prices1 = get_price_df(long_short_positions, returns, model_type='ML')
        last_index1 = prices1.shape[0] - 1
        p_last1 = prices1[last_index1]
        stats1['Total Return'] = (p_last1 - prices1[0]) / prices1[0]
    elif model_type =='LSTM':
        prices1 = get_price_df(long_short_positions, returns, model_type='LSTM')
        last_index1 = prices1.shape[0] - 1
        p_last1 = prices1[last_index1]
        stats1['Total Return'] = (p_last1 - prices1[0]) / prices1[0]
    elif model_type =='13':
        prices1 = get_price_df(long_short_positions, returns, model_type='13')
        last_index1 = prices1.shape[0] - 1
        p_last1 = prices1[last_index1]
        stats1['Total Return'] = (p_last1 - prices1[0]) / prices1[0]

    stats1['Avg Monthly Return'] = (1 + stats1['Total Return']) ** (1 / len(prices1)) - 1
    stats1['Avg Mkt Cap'] = (long_short_positions.replace(-1, 1) * df_market_cap).replace(0, np.nan).mean(axis=1).mean()
    stats1['Fraction>0'] = long_short_sum[long_short_sum > 0].count() / len(long_short_sum)
    stats1['+ Return Stocks (Avg)'] = long_short_returns[long_short_returns > 0].mean(axis=1).mean()
    stats1['- Return Stocks (Avg)'] = long_short_returns[long_short_returns < 0].mean(axis=1).mean()
    stats1['Odds Ratio'] = long_short_returns[long_short_returns > 0].count(axis=1).sum() / long_short_returns[long_short_returns != 0].count(axis=1).sum()
    stats1['Odds Ratio 98-10'] = long_short_returns_98_10[long_short_returns_98_10 > 0].count(axis=1).sum() / long_short_returns_98_10[long_short_returns_98_10 != 0].count(axis=1).sum()
    stats1['Odds Ratio 11-22'] = long_short_returns_11_22[long_short_returns_11_22 > 0].count(axis=1).sum() / long_short_returns_11_22[long_short_returns_11_22 != 0].count(axis=1).sum()
    stats1['Transactions/Month'] = df_transactions.mean(axis=1)[0]


    if model_type is None:
        prices2 = get_price_df(long_positions, returns, model_type=None)
        last_index2 = prices2.shape[0] - 1
        p_last2 = prices2[last_index2]
        stats2['Total Return'] = (p_last2 - prices2[0]) / prices2[0]
    elif model_type == 'ML':
        prices2 = get_price_df(long_positions, returns, model_type='ML')
        last_index2 = prices2.shape[0] - 1
        p_last2 = prices2[last_index2]
        stats2['Total Return'] = (p_last2 - prices2[0]) / prices2[0]
    elif model_type =='LSTM':
        prices2 = get_price_df(long_positions, returns, model_type='LSTM')
        last_index2 = prices2.shape[0] - 1
        p_last2 = prices2[last_index2]
        stats2['Total Return'] = (p_last2 - prices2[0]) / prices2[0]
    elif model_type =='13':
        prices2 = get_price_df(long_positions, returns, model_type='13')
        last_index2 = prices2.shape[0] - 1
        p_last2 = prices2[last_index2]
        stats2['Total Return'] = (p_last2 - prices2[0]) / prices2[0]

    stats2['Avg Monthly Return'] = (1 + stats2['Total Return']) ** (1 / len(prices2)) - 1
    stats2['Avg Mkt Cap'] = (long_positions * df_market_cap).replace(0, np.nan).mean(axis=1).mean()
    stats2['Fraction>0'] = long_sum[long_sum > 0].count() / len(long_sum)
    stats2['+ Return Stocks (Avg)'] = long_returns[long_returns > 0].mean(axis=1).mean()
    stats2['- Return Stocks (Avg)'] = long_returns[long_returns < 0].mean(axis=1).mean()
    stats2['Odds Ratio'] = long_returns[long_returns > 0].count(axis=1).sum() / long_returns[long_returns != 0].count(axis=1).sum()
    stats2['Odds Ratio 98-10'] = long_returns_98_10[long_returns_98_10 > 0].count(axis=1).sum() / long_returns_98_10[long_returns_98_10 != 0].count(axis=1).sum()
    stats2['Odds Ratio 11-22'] = long_returns_11_22[long_returns_11_22 > 0].count(axis=1).sum() / long_returns_11_22[long_returns_11_22 != 0].count(axis=1).sum()
    stats2['Transactions/Month'] = df_transactions_1.mean(axis=1)[0]


    if model_type is None:
        prices3 = get_price_df(short_positions, returns, model_type=None)
        last_index3 = prices3.shape[0] - 1
        p_last3 = prices3[last_index3]
        stats3['Total Return'] = (p_last3 - prices3[0]) / prices3[0]
    elif model_type == 'ML':
        prices3 = get_price_df(short_positions, returns, model_type='ML')
        last_index3 = prices3.shape[0] - 1
        p_last3 = prices3[last_index3]
        stats3['Total Return'] = (p_last3 - prices3[0]) / prices3[0]
    elif model_type == 'LSTM':
        prices3 = get_price_df(short_positions, returns, model_type='LSTM')
        last_index3 = prices3.shape[0] - 1
        p_last3 = prices3[last_index3]
        stats3['Total Return'] = (p_last3 - prices3[0]) / prices3[0]
    elif model_type == '13':
        prices3 = get_price_df(short_positions, returns, model_type='13')
        last_index3 = prices3.shape[0] - 1
        p_last3 = prices3[last_index3]
        stats3['Total Return'] = (p_last3 - prices3[0]) / prices3[0]

    stats3['Avg Monthly Return'] = (1 + stats3['Total Return']) ** (1 / len(prices2)) - 1
    stats3['Avg Mkt Cap'] = (short_positions.replace(-1, 1) * df_market_cap).replace(0, np.nan).mean(axis=1).mean()
    stats3['Fraction>0'] = short_sum[short_sum > 0].count() / len(short_sum)
    stats3['+ Return Stocks (Avg)'] = short_returns[short_returns > 0].mean(axis=1).mean()
    stats3['- Return Stocks (Avg)'] = short_returns[short_returns < 0].mean(axis=1).mean()
    stats3['Odds Ratio'] = short_returns[short_returns > 0].count(axis=1).sum() / short_returns[short_returns != 0].count(axis=1).sum()
    stats3['Odds Ratio 98-10'] = short_returns_98_10[short_returns_98_10 > 0].count(axis=1).sum() / short_returns_98_10[short_returns_98_10 != 0].count(axis=1).sum()
    stats3['Odds Ratio 11-22'] = short_returns_11_22[short_returns_11_22 > 0].count(axis=1).sum() / short_returns_11_22[short_returns_11_22 != 0].count(axis=1).sum()
    stats3['Transactions/Month'] = df_transactions_2.mean(axis=1)[0]

    return pd.concat([stats1, stats2, stats3], axis=0)